In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

from sklearn import neighbors
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
prop = pickle.load(open("../../datos/Data.p","rb"))

# Nos quedamos con la parte del set de datos que nos importa

In [3]:
p = prop[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos']]
p.dropna(inplace = True)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Separamos la Latitud-Longitud

In [4]:
p['lat'] = p.apply(lambda row: row[5][0], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
p['lon'] = p.apply(lambda row: row[5][1], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos']]

# Asignamos un valor numerico al tipo de propiedad

In [7]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [8]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

# Asignamos un valor a los barrios

In [9]:
def asignarPV(name, dic):
    return dic[name]

In [10]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [11]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*10
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [12]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [13]:
p = p[['property_type','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos', 'place_value']]

# KNN

In [14]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(p[['property_type','surface_total_in_m2','surface_covered_in_m2',
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos','Cant_LugaresTuristicos',
        'place_value']],p[['price_usd_per_m2']],test_size=0.2)

In [15]:
columnas = ['surface_total_in_m2','surface_covered_in_m2',\
        'Cant_ColeYUniv','Cant_LocalesGastronomicos',\
        'Cant_LugaresTuristicos']

In [16]:
#Parametros modificables de knn
lista_distancias = [1,2] #1 es manhattan, 2 euclidea
lista_vecinos = [x for x in xrange(1,10)]
lista_algoritmos = ['auto', 'ball_tree', 'kd_tree']

In [17]:
aux = Ytest.copy()

In [28]:
res = []
for k in lista_vecinos:
    for d in lista_distancias:
        for a in lista_algoritmos:
            aux = Ytest.copy()
            #Inicializo la instancia del algoritmo
            knn = neighbors.KNeighborsRegressor(n_neighbors = k, p = d, algorithm = a)
            knn.fit(Xtrn, Ytrn['price_usd_per_m2'])
            aux['predict'] = knn.predict(Xtest)

            error = mean_squared_error(Ytest['price_usd_per_m2'] , aux['predict'])

            res.append( (error, k, d, a) )

In [35]:
r = sorted(res, key=lambda x: x[0])

In [39]:
#aca podemos ver una busqueda aproximada de los parametros.
r

[(6436266.2916668355, 4, 1, 'ball_tree'),
 (6461835.3112882609, 4, 2, 'ball_tree'),
 (6465733.3134354223, 4, 1, 'auto'),
 (6465733.3134354223, 4, 1, 'kd_tree'),
 (6493949.3622172996, 4, 2, 'auto'),
 (6493949.3622172996, 4, 2, 'kd_tree'),
 (6494809.8206934752, 3, 2, 'ball_tree'),
 (6522362.3303189902, 5, 1, 'ball_tree'),
 (6528066.5339958426, 3, 1, 'ball_tree'),
 (6529832.9857137566, 3, 2, 'auto'),
 (6529832.9857137566, 3, 2, 'kd_tree'),
 (6530415.9345181547, 5, 2, 'ball_tree'),
 (6547085.7892487179, 5, 2, 'auto'),
 (6547085.7892487179, 5, 2, 'kd_tree'),
 (6557496.1229593512, 5, 1, 'auto'),
 (6557496.1229593512, 5, 1, 'kd_tree'),
 (6561822.8279918656, 3, 1, 'auto'),
 (6561822.8279918656, 3, 1, 'kd_tree'),
 (6569078.7695191195, 7, 1, 'ball_tree'),
 (6586194.1670135204, 6, 1, 'ball_tree'),
 (6596517.2857652893, 7, 1, 'auto'),
 (6596517.2857652893, 7, 1, 'kd_tree'),
 (6635148.0304942969, 6, 1, 'auto'),
 (6635148.0304942969, 6, 1, 'kd_tree'),
 (6645129.5465439223, 7, 2, 'ball_tree'),
 (6646